# Name : Omkar Thorve
# PRN: 24070149016

# Pip install OpenCV

In [1]:
pip install opencv-contrib-python --upgrade 

Note: you may need to restart the kernel to use updated packages.


In [1]:
import cv2

# Class Code for video 

In [3]:
# class code
cam = cv2.VideoCapture(0)

frame_width = int(cam.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cam.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('Documents/SEM 2/Machine Vision',fourcc,20.0,(frame_width,frame_height))

while True:
    ret, frame = cam.read()
    out.write(frame)
    cv2.imshow('Camera',frame)
    if cv2.waitKey(1) == ord('q'):
        break
cam.release()
out.release()
cv2.destroyAllWindows()

[ WARN:0@4.523] global cap.cpp:645 open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.10.0) /Users/runner/miniforge3/conda-bld/libopencv_1735821636510/work/modules/videoio/src/cap_images.cpp:430: error: (-215:Assertion failed) !filename_pattern.empty() in function 'open'




# Code for object detection in an image

In [ ]:
# image object detection
import numpy as np
def detect_objects_in_image(image_path):
    
    image = cv2.imread(image_path)
    output_image = image.copy()
    
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    
    for (x, y, w, h) in faces:
        cv2.rectangle(output_image, (x, y), (x+w, y+h), (255, 0, 0), 2)
        cv2.putText(output_image, 'Face', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) if len(image.shape) == 3 else image
    _, binary = cv2.threshold(gray, 180, 255, cv2.THRESH_BINARY_INV)
    
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > 500:  
            x, y, w, h = cv2.boundingRect(contour)
            aspect_ratio = w/h
            if 0.5 <= aspect_ratio <= 15:  
                cv2.rectangle(output_image, (x, y), (x+w, y+h), (0, 255, 0), 2)
                cv2.putText(output_image, 'Text', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    

    cv2.imshow('Result', output_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    
    cv2.imwrite('Documents/SEM 2/Machine Vision/Screenshot 2025-01-08 at 12.16.28 PM.png', output_image)


detect_objects_in_image('Documents/SEM 2/Machine Vision/Screenshot 2025-01-08 at 12.16.28 PM.png')

# Code for object detection from Web Cam video

In [ ]:
# webcam video object detect
import numpy as np
import time

def process_video_detection(video_source=0):
  
    # Initialize video capture
    cap = cv2.VideoCapture(video_source)
    
    # Check if video opened successfully
    if not cap.isOpened():
        print("Error: Could not open video source")
        return
    
    # Load the pre-trained face detection classifier
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    
    # frame dimension and FPS
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    # saves processed video
    output_video = cv2.VideoWriter('detected_output.avi', 
                                 cv2.VideoWriter_fourcc(*'XVID'),
                                 fps, (frame_width, frame_height))
    
    # FPS calculation variables
    start_time = time.time()
    frame_count = 0
    fps_display = 0
    
    while True:
        # Read frame from video
        ret, frame = cap.read()
        
        if not ret:
            print("End of video stream")
            break
            
        # Create a copy for drawing
        output_frame = frame.copy()
        
        # Convert to grayscale for detection
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Face Detection
        faces = face_cascade.detectMultiScale(gray, 
                                            scaleFactor=1.1,
                                            minNeighbors=5,
                                            minSize=(30, 30))
        
        # Draw rectangles around faces
        for (x, y, w, h) in faces:
            cv2.rectangle(output_frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            cv2.putText(output_frame, 'Face', (x, y-10),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
        
        # Text Region Detection
        # Apply threshold to get text regions
        _, binary = cv2.threshold(gray, 180, 255, cv2.THRESH_BINARY_INV)
        
        # Find contours in the binary image
        contours, _ = cv2.findContours(binary, 
                                     cv2.RETR_EXTERNAL,
                                     cv2.CHAIN_APPROX_SIMPLE)
        
        # Process each contour for text detection
        for contour in contours:
            area = cv2.contourArea(contour)
            if area > 500:  # Filter small contours
                x, y, w, h = cv2.boundingRect(contour)
                aspect_ratio = w/h
                # Filter based on aspect ratio to reduce false positives
                if 0.5 <= aspect_ratio <= 15:
                    cv2.rectangle(output_frame, (x, y), (x+w, y+h),
                                (0, 255, 0), 2)
                    cv2.putText(output_frame, 'Text', (x, y-10),
                              cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        
        # Calculate and display FPS
        frame_count += 1
        if frame_count % 30 == 0:  # Update FPS every 30 frames
            end_time = time.time()
            fps_display = frame_count / (end_time - start_time)
        
        # Display FPS on frame
        cv2.putText(output_frame, f'FPS: {fps_display:.2f}',
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX,
                    1, (0, 0, 255), 2)
        
        # Write frame to output video
        output_video.write(output_frame)
        
        # Display the processed frame
        cv2.imshow('Video Detection', output_frame)
        
        # Break loop on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Clean up
    cap.release()
    output_video.release()
    cv2.destroyAllWindows()

def main():
    # Video source 
    video_source = 0
    process_video_detection(0)

if __name__ == "__main__":
    main()
